# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

In the CSV file given to us by John from finances, a large table can be found. 

Each row represents a transaction from a sale of gadgets to a company. 

The file is loaded to get more information.

In [1]:
import pandas as pd
import re
import requests

In [2]:
df = pd.read_csv('transactions.csv', encoding='utf-8-sig')
df.head(3)

,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30


In [3]:
start_rows = len(df)
start_rows

20568

The table consists of 20568 data rows and there are 6 columns of information for each transaction. 

The first column, 'part', is not unique nor does it give information important in regards to doing this exercise. Therefore, it is removed from the table.

In [4]:
df.drop(['part'], inplace=True, axis=1)
df.head(3)

,company,country,city,price,date
0,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30


To gain a better knowledge of the type of data, the first order of business is to have python tell us what type of data we are working with. 

In [5]:
df.dtypes

company    object
country    object
city       object
price      object
date       object
dtype: object

The above output tells us that the data in all the columns is of 'object' type - also known as 'string' type. However, from the table it is clear that the data in the 'price' and 'date' columns should not be an object but rather a float and datetime, respectively. 

Now when we know what the data is, we need to make a plan for how to clean it:

- Make sure that all dates in the 'date' column are real dates and in the same format.
- Remove all the NaN values from company, country and city.
- Clean the 'price' column by removing rows without a price and make sure all currencies are in euro. 
- Remove duplicates.

#### Make sure that all dates in the 'date' column are real dates and in the same format.

***I:*** We find the transactions with the default date format. If a transaction is not in the default format it is changed to it. Further, if a date is a non-valid date it is removed (e.g. 2017-13-32). 

In [6]:
out_of_range = []

for i in range(0,len(df)):
    try:
        pd.to_datetime(df.iloc[i,4], format='%Y-%m-%d')
    except:
        try: 
            df.iloc[i,4] = pd.to_datetime(df.iloc[i,4])
        except:
            out_of_range.append(i)

df.drop(out_of_range, inplace=True)

***II:*** We change the dtype from object to datetime64. 

In [7]:
df['date'] = df['date'].astype('datetime64[ns]')
df.dtypes

company            object
country            object
city               object
price              object
date       datetime64[ns]
dtype: object

***SUCCES!*** All dates are now changed to the same format and the dtype is changed to datetime64.

#### Remove all the NaN values from company, country and city.

***I:*** We want to make sure that all countries in the country column is real countries. To do this we extract an updated json-file from the worldbank data and collect all real countries in a list.

In [8]:
r = requests.get('http://api.worldbank.org/v2/sources/2/country/all/data?per_page=500&format=json&mrnev=1')
json_response = r.json()
res = json_response['source']

list_country = []

for r in res:    
    inner_res = r.get('concept')
    for r in inner_res:
        country = r.get('variable')
        for r in country:
            country = r.get('value')
            list_country.append(country)

Now after we generated the list it is easy to see which country is not in the list and thus is suspicious. These should be taken a closer look at.

In [9]:
not_real = []
for i in range(0,len(df)):
    if df.iloc[i,1] not in list_country:
        not_real.append(df.iloc[i,1])

print(list(set(not_real)))

[nan, 'Tyskland', 'Portuga', 'US']


We quickly realize that John from finances made some typos here and manually change these to the correct value.

In [10]:
for i in range(0,len(df)):
    if df.iloc[i,1] == 'US':
        df.iloc[i,1] = 'United States'
    elif df.iloc[i,1] == 'Tyskland':
        df.iloc[i,1] = 'Germany'
    elif df.iloc[i,1] == 'Portuga':
        df.iloc[i,1] = 'Portugal'

When John from finances clearly could make typos in the country-column we now wonder whether he also can do the same thing in the company-column. Since there is not database over imagnianary comapnies we have to do something else. Instead we count the number of times the company occurs in the data frame and make the assumption that any company with a representation below or equal to 5 have to be looked closelyer upon. 

In [11]:
counts = dict()
weird_company = []

for i in range(0,len(df)):
    if df.iloc[i,0] in counts:
        counts[df.iloc[i,0]] += 1
    else:
        counts[df.iloc[i,0]] = 1    

for char in sorted(counts.keys()):
    if counts[char] <= 5:
        weird_company.append(char)

print('Companies that is underrepresented: %s' % weird_company)

Companies that is underrepresented: [' -', ' a', 'Laj0', 'Ntagz', 'Thoughtmixz', 'Zooxo.', 'aa']


After a quick glance at the 'true' companies we see that trully enough most of these were typos. These are corrected. Three companies still look weird (' -', ' a' and 'aa') and we wonder why. We will leave them be for now. 

In [12]:
for i in range(0,len(df)):
    if df.iloc[i,0] == 'Zooxo.':
        df.iloc[i,0] = 'Zooxo'
    elif df.iloc[i,0] == 'Thoughtmixz':
        df.iloc[i,0] = 'Thoughtmix'
    elif df.iloc[i,0] == 'Ntagz':
        df.iloc[i,0] = 'Ntags'
    elif df.iloc[i,0] == 'Laj0':
        df.iloc[i,0] = 'Lajo' 

Now we group each company by its countries. We assume that three thing will happen now. (I): There is only one country were this company is present. In that case nothing is done. (II): There is one country and one NaN value for the company. Here we replace the NaN value with the specific country. (III): There are multiple countries were this country is present (and possible also NaN values). This is an odd case and we have to look closer at this.

In [13]:
grouped = df.groupby('company')['country'].unique().apply(list).to_dict()

still_weird_company = []

for i in range(0,len(df)):
    if pd.isna(df.iloc[i,1]) == True:
        if len(grouped[df.iloc[i,0]]) == 2:
            df.iloc[i,1] = grouped[df.iloc[i,0]][0]
        elif len(grouped[df.iloc[i,0]]) >= 2:
            still_weird_company.append(df.iloc[i,0])
            
print('Countries that has to be investigated further: %s' % list(set(still_weird_company)))

Countries that has to be investigated further: ['Flipstorm']


***SUCCES!*** All companies (except 'Flipstorm') now only have one country and no NaN value should be present.

Now we group each company by its countries and city. We assume that three thing will happen now. (I): There is only one country and one city were this company is present. In that case nothing is done. (II): There is one country, one city and one NaN value for the company. In that case nothing is done just yet. (III): There are multiple countries or multiple cities were this country is present (and possible also NaN values). This is an odd case and we have to look closer at this.

In [14]:
grouped = (df.groupby(['company'])['country', 'city']
       .apply(lambda x: pd.unique(x.values.ravel()).tolist()))

weird_city = []

for i in range(0,len(df)):
    if len(grouped[df.iloc[i,0]]) > 3:
        if grouped[df.iloc[i,0]] in weird_city:
            pass
        else:
            weird_city.append(grouped[df.iloc[i,0]])

print(weird_city)

[['Greece', 'Athens', 'France', 'Nanterre', nan], ['Portugal', 'Braga', nan, 'Monção'], ['Portugal', 'Amadora\t', 'Vila Fria', nan]]


We see three cases that is weird. But after a quick google search we realize that John from finances once again made a mistake. Vila Fria is not a city but an area in Amadora and thus we changed it to Amadora. Monção is neither a city by an area as well. Monção is thus changed to Braga. Note: We also removed the extra /t from Amadora. 

Furthermore; If the country is Greece the city should be Athens, and vice versa. If the country is France the city should be Nanterre, and vice versa. 

In [15]:
for i in range(0,len(df)):
    if df.iloc[i,2] == 'Amadora\t':
        df.iloc[i,2] = 'Amadora'
    elif df.iloc[i,2] == 'Vila Fria':
        df.iloc[i,2] = 'Amadora'
    elif df.iloc[i,2] == 'Monção':
        df.iloc[i,2] = 'Braga'
        
    elif df.iloc[i,1] == 'Greece':
        df.iloc[i,2] = 'Athens'
    elif df.iloc[i,1] == 'France':
        df.iloc[i,2] = 'Nanterre'
    elif df.iloc[i,2] == 'Athens':
        df.iloc[i,1] = 'Greece'
    elif df.iloc[i,2] == 'Nanterre':
        df.iloc[i,1] = 'France'

Now that you fixed this, you once again group them together. This time you focus on all the cases were there is one country, one city and one NaN value for the company. Here we replace the NaN value with the specific city since we now the NaN value cannot be the country.

In [16]:
grouped = (df.groupby(['company'])['country', 'city']
       .apply(lambda x: pd.unique(x.values.ravel()).tolist()))

for i in range(0,len(df)):
    if pd.isna(df.iloc[i,2]) == True:
        if len(grouped[df.iloc[i,0]]) == 3:
            df.iloc[i,2] = grouped[df.iloc[i,0]][1]

A sight of relieve leaves your body. 'Done' you think. But suddenly you remeber the three weird companies 'aa', ' a' and ' -'. You wonder if John's typo is so servere that it actually is not a mistake. To check this, you see if the placement of the company is identical to some of the other companies.

In [17]:
grouped = (df.groupby(['company'])['country', 'city']
       .apply(lambda x: pd.unique(x.values.ravel()).tolist()))

weird_company = ['aa',' a',' -']

for j in weird_company:
    lol = grouped[j]
    for i in range(len(grouped)):
        if grouped[i] == lol and grouped.keys()[i] not in weird_company:
            print('The company placement of "%s" matches the company "%s".' % (j, grouped.keys()[i]))

The company placement of "aa" matches the company "Wordify".
The company placement of " a" matches the company "Wordify".
The company placement of " -" matches the company "Zoonder".


Each companies location only matches one other company and we thus make the assumption that these companies are indeed the same as the once at the same location. 

In [18]:
for i in range(0,len(df)):
    if df.iloc[i,0] == 'aa':
        df.iloc[i,0] = 'Wordify'
    elif df.iloc[i,0] == ' a':
        df.iloc[i,0] = 'Wordify'
    elif df.iloc[i,0] == ' -':
        df.iloc[i,0] = 'Zoonder'

We now do a final status overview to check if everything looks good.

In [19]:
grouped = (df.groupby(['company'])['country', 'city']
       .apply(lambda x: pd.unique(x.values.ravel()).tolist()))
grouped

company
Avaveo                           [France, Nanterre]
Brainsphere                       [Portugal, Braga]
Bubblemix                            [Japan, Asaka]
Buzzbean                      [Germany, Düsseldorf]
Chatterbridge                    [Spain, Barcelona]
Eimbee                           [France, Nanterre]
Flipstorm        [Greece, Athens, France, Nanterre]
Gabcube                          [Portugal, Almada]
Gabtune                          [France, Nanterre]
Gevee                            [France, Nanterre]
Innojam                    [Netherlands, Amsterdam]
Kanoodle                           [Japan, Niihama]
Lajo                               [Greece, Athens]
Ntags                            [Portugal, Lisbon]
Realpoint                        [Portugal, Lisbon]
Rhycero                          [France, Nanterre]
Riffpath                           [Greece, Athens]
Roodel                          [Portugal, Aranhas]
Shufflebeat                       [Portugal, Porto]
Tagt

A weird feeling creep upon you. Yesterday in the news you saw that companies cheated in taxes by placing there company in non-existing cities. GASP! To ease you fear you quickly make a script that can check if the respective city is placed in the correct land. 

In [20]:
r = requests.get('https://raw.githubusercontent.com/russ666/all-countries-and-cities-json/6ee538beca8914133259b401ba47a550313e8984/countries.json')
json_response = r.json()
res = json_response
res

weird_countries = {}

country_list = {}

for r in res:
    country_list[r] = res[r]
    
for i in range(0,len(df)):
    if df.iloc[i,1] in country_list.keys():
        country = str(df.iloc[i,1])
        if df.iloc[i,2] not in country_list[country]:
            weird_countries[df.iloc[i,1]] = df.iloc[i,2]

print(weird_countries)

{'Germany': 'Düsseldorf', 'Portugal': 'Aranhas', 'Switzerland': 'Zürich'}


After quick look (and a google search) you mind is at rest again. All the above is true city in its respective country.

***SUCCES!*** Now there are no more NaN values.

#### OLD TEXT!!! :

We see that the dream scenario of having a company in only one country and one city is far from fulfilled. 

***V:*** Three companies, 'aa', ' a' and ' -', seem suspicious. For each of these companies it is clear that only one other company has the exact same location. Tus, we assume that the three suspicious companies are actually typos of the companies with the same location.

***VI:*** We see that the company 'Brainsphere' is in two different cities in Portugal. So we count how many times the company occurs in each city.

***VII:*** It is now clear that John from finances made a mistake since 'Brainspehere' is only found once in Monção. So we change the city from Monção to Braga.

***VIII:*** We see that the company 'Thoughtmix' is in two different cities in Portugal. But after a quick google search we realize that John from finances once again made a mistake. Vila Fria is not a city but an area in Amadora and thus we changed it to Amadora. Note: We also removed the extra /t from Amadora. 

***IX:*** Most of the companies now only have one specific country, one specific city and a NaN value in the 'city' column (except the company 'Flipstorm'). All the NaN values are switched to the specific city.

***X:*** Now only the company 'Flipstorm' is messed up. This is easiely fixed. If the country is Greece the city should be Athens, and vice versa. If the country is France the city should be Nanterre, and vice versa. 

#### Clean the 'price' column by removing rows without a price and make sure all currencies are in euro.

In [21]:
r = requests.get('https://api.exchangeratesapi.io/history?start_at=2000-01-01&end_at=2019-10-08')
json_response = r.json()

rates = json_response['rates']

real_dates = []

for dates in rates:
    real_dates.append(dates)

In [22]:
removable = []

for i in range(0,len(df)):
    price = str(df.iloc[i,3])
    
    regex = re.search(r"^\-?\d+\.\d{,3}\€{1}", price)
    
    if regex is None:
        regex_new = re.search(r"\-?\d+\.\d{,3}", price)
        
        if regex_new is None:
            removable.append(price)
            
list(set(removable))

['-', 'nan', 'na', 'void']

In [23]:
remove_price = []
for index, row in df.iterrows():
    if pd.isna(row['price']) == True or row['price'] == '-' or row['price'] == 'na' or row['price'] == 'void':
        remove_price.append(index)
df.drop(remove_price, inplace = True)

In [24]:
r = requests.get('https://gist.githubusercontent.com/Fluidbyte/2973986/raw/b0d1722b04b0a737aade2ce6e055263625a0b435/Common-Currency.json')
json_response = r.json()

aDict = {}
for r in json_response: 
    aDict[json_response[r]['symbol']] = r

In [25]:
no_currency = []

for i in range(0,len(df)):
    try:
        original_price = df.iloc[i,3]

        regex = re.search(r"^\-?\d+\.\d{,3}\€{1}", original_price)

        if regex is not None:
            price = float(df.iloc[i,3][:-1])
            df.iloc[i,3] = round(price,2)

        elif regex is None:
            
            non_digit = re.search(r'\D',df.iloc[i,3])
            non_digit = non_digit.group(0)
            
            date = str(df.iloc[i,4])
            date = date[:-9]

            currency = aDict[non_digit]
            
            while date not in real_dates:
                date = pd.to_datetime(date)
                date = date - pd.Timedelta(days=1)
                date = str(date)
                date = date[:-9]
                
            
            rate = rates[date][currency]
            
  
            price = float(df.iloc[i,3][1:]) / rate
            df.iloc[i,3] = round(price,2)  
            
    except:
        no_currency.append(i)

In [26]:
df.drop(no_currency, inplace=True)

In [27]:
df['price'] = pd.to_numeric(df['price'])
df.dtypes

company            object
country            object
city               object
price             float64
date       datetime64[ns]
dtype: object

#### OLD TEXT:

***II:*** We change the currency to euro in lines with '£', '$' and '¥' and for all the prices the currency symbol is removed.

Websites used for currency:

- Currency for £ to €: https://www.xe.com/currencyconverter/convert/?Amount=1&From=GBP&To=EUR visited: 3/10
- Currency for $ to €: https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=EUR visited: 3/10
- Currency for ¥ to €: https://www.xe.com/currencyconverter/convert/?Amount=1&From=JPY&To=EUR visited: 3/10

***III***: We delete the lines with 'void', 'na', 'nan' and '-'

***IV***: The data in the 'price' column is changed to float64.

***SUCCESS!*** The currencies have been changed to euro and the data type is now float. 

#### Remove duplicates.

In [28]:
df = df.drop_duplicates()

#### Status.

The data has been cleaned and is now ready for further analysis. 

In [29]:
end_rows = len(df)
start_rows - end_rows

17

In total, 17 rows have been removed from the table.

---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

To enrich our data set we have contructed a new CSV file with more columns of data. 

This data has been derived from The World Bank. 

## MANGLER MERE INFO: ARGUE, where is the data from etc.

## QUESTION: you can not ask the reader of your notebook to download any data manually.???? ER DET FORKERT DET VI HAR GJORT?

In [30]:
r = requests.get('http://api.worldbank.org/v2/sources/2/country/all/series/NY.GDP.MKTP.CD/data?per_page=500&format=json&mrnev=1')
json_response = r.json()
res = json_response['source']['data']

In [31]:
list_country = []
list_value_GDP_dollar = []

counter = 0
for r in res:
    inner_res = r.get('variable')
    value = r.get('value')

    list_value_GDP_dollar.append(value)
    
    for r in inner_res:
        country = r.get('value')
        list_country.append(country)
        break

unique_country = []
for country in list_country:
    if country not in unique_country:
        unique_country.append(country)

In [32]:
r = requests.get('https://api.exchangeratesapi.io/latest?symbols=USD')
json_response = r.json()

rates = json_response['rates']['USD']

In [33]:
list_value_GDP_euro = []
for i in list_value_GDP_dollar:
    list_value_GDP_euro.append(i*rates)

In [34]:
new_df = pd.DataFrame(list(zip(unique_country,list_value_GDP_euro)), columns =['country','gdp_euro'])
print(new_df.head(3))

                          country      gdp_euro
0                      Arab World  3.055854e+12
1          Caribbean small states  8.012120e+10
2  Central Europe and the Baltics  1.792142e+12


In [35]:
df = pd.merge(df, new_df, on='country', how='inner')
print(df.head(3))

         company country       city   price                date      gdp_euro
0  Chatterbridge   Spain  Barcelona  784.79 2016-01-02 00:01:05  1.566811e+12
1  Chatterbridge   Spain  Barcelona  412.55 2016-01-02 04:51:55  1.566811e+12
2  Chatterbridge   Spain  Barcelona  359.52 2016-01-02 07:20:59  1.566811e+12


In [36]:
df.dtypes

company             object
country             object
city                object
price              float64
date        datetime64[ns]
gdp_euro           float64
dtype: object

In [37]:
len(df)

20551

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

In [38]:
import sqlite3

In [39]:
conn = sqlite3.connect('transactions.sqlite')

In [40]:
c = conn.cursor()

In [41]:
df.to_sql("transactions", conn, if_exists='replace')

In [42]:
conn.commit()

In [43]:
c.execute('SELECT date(date) FROM transactions')

In [44]:
len(c.fetchall())

20551

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

In [45]:
sort_by_price = df.sort_values('price')
sort_by_price

,company,country,city,price,date,gdp_euro
8810,Zooxo,United Kingdom,London,-727.47,2016-06-28 06:31:55,3.103773e+12
901,Chatterbridge,Spain,Barcelona,-521.70,2017-09-09 04:05:49,1.566811e+12
15933,Ntags,Portugal,Lisbon,-364.17,2018-01-27 14:43:38,2.614437e+11
12214,Thoughtmix,Portugal,Amadora,-297.24,2016-09-22 18:01:17,2.614437e+11
9808,Zooxo,United Kingdom,London,-283.85,2018-04-18 03:51:50,3.103773e+12
3440,Flipstorm,France,Nanterre,-217.88,2016-01-05 12:21:25,3.051400e+12
2547,Yozio,Greece,Athens,-185.05,2016-11-07 08:16:26,2.395298e+11
38,Chatterbridge,Spain,Barcelona,-181.47,2016-01-25 09:38:31,1.566811e+12
12463,Ntags,Portugal,Lisbon,-173.53,2016-10-23 14:27:46,2.614437e+11
12994,Shufflebeat,Portugal,Porto,-168.28,2017-01-02 12:09:55,2.614437e+11


---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---